In [1]:
if True:
    import os
    os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'
    execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))
    import os
    import sys
else:
    os.environ['SPARK_HOME'] = '/usr/lib/spark'
    sys.path.insert(0, '/usr/lib/spark/python/lib/py4j-0.9-src.zip')
    sys.path.insert(0, '/usr/lib/spark/python/')
    sys.path.insert(0, '/usr/local/lib64/python2.7/site-packages')
    sys.path.insert(0,'/usr/local/lib/python2.7/site-packages')
    
    from pyspark import SparkContext
    from pyspark.sql import SQLContext, HiveContext
    
    try: sc = SparkContext()
    except: None    
    sqlc = SQLContext(sc)
    spark = sqlc

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.0.2
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkSession available as 'spark'.


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import seaborn as sns

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
import numpy as np
import pandas as pd
import time
import json

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, MapType
import pyspark.sql.functions as F

In [5]:
from operator import add
from pyspark.sql.types import *

In [6]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
from datetime import datetime

# Load data

In [7]:
strain_predictions = spark.read.parquet("lab_12/cache/strain_bp")


In [8]:
strain_predictions.show(2)

+------+-------+------+--------------+--------------+-----------+-----------+------------------+-------------------+
|userId|movieId|rating|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|         bp_rating|      bp_rating_err|
+------+-------+------+--------------+--------------+-----------+-----------+------------------+-------------------+
|170647|     31|   4.5|           179|           109|     unr_51|    inr_101|4.2530035613100505|0.24699643868994947|
|198447|     31|   2.0|            53|           109|     unr_51|    inr_101| 3.847971875157315|-1.8479718751573149|
+------+-------+------+--------------+--------------+-----------+-----------+------------------+-------------------+
only showing top 2 rows



In [9]:
stest_predictions = spark.read.parquet("lab_12/cache/stest_bp")

In [10]:
stest_predictions.show(2)

+------+-------+------+--------------+--------------+-----------+-----------+-----------------+-------------------+
|userId|movieId|rating|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|        bp_rating|      bp_rating_err|
+------+-------+------+--------------+--------------+-----------+-----------+-----------------+-------------------+
|227044|     31|   3.5|          20.0|         109.0|  unr_06_50|    inr_101|4.036272701087659|-0.5362727010876593|
| 38648|     31|   4.0|         225.0|         109.0|     unr_51|    inr_101|4.524626216198095|-0.5246262161980946|
+------+-------+------+--------------+--------------+-----------+-----------+-----------------+-------------------+
only showing top 2 rows



# ALS on full score

## Prepeare train,validate,test data

In [14]:
strain_predictions.rdd.take(1)

[Row(userId=170647, movieId=31, rating=4.5, n_user_ratings=179, n_item_ratings=109, nr_user_cat=u'unr_51', nr_item_cat=u'inr_101', bp_rating=4.2530035613100505, bp_rating_err=0.24699643868994947)]

In [15]:
def convert_to_als_rating(r):
    return Rating(r.userId, r.movieId, r.rating)

In [16]:
strain_ratings = strain_predictions.rdd.map(convert_to_als_rating)

In [17]:
strain_ratings.first()

Rating(user=170647, product=31, rating=4.5)

# Train

In [18]:
rank = 25
numIterations = 15
lambda_ = 0.1

In [19]:
%%time
model = ALS.train(strain_ratings, rank, numIterations, lambda_=lambda_, seed=68)

CPU times: user 36 ms, sys: 8 ms, total: 44 ms
Wall time: 1min 57s


# Predict on test

In [20]:
stest_raw = model.predictAll(stest_predictions.rdd.map(lambda x: (x.userId, x.movieId)))

In [21]:
stest_raw.first()

Rating(user=185012, product=11817, rating=4.442669416536464)

In [22]:
stest_als_pred = stest_raw.toDF(schema=StructType([
                                            StructField("userId",  IntegerType()),
                                            StructField("movieId", IntegerType()),
                                            StructField("predicted_als_rating",  FloatType())]))

In [23]:
stest_als_pred.show(3)

+------+-------+--------------------+
|userId|movieId|predicted_als_rating|
+------+-------+--------------------+
|185012|  11817|           4.4426694|
|185012|  11717|            3.726664|
|185012|  20945|            3.527127|
+------+-------+--------------------+
only showing top 3 rows



In [ ]:
#stest_als_pred.coalesce(8).write.parquet("lab_12/cache/stest_als25_has_missed", 
#                                         mode='overwrite')

# Eval

In [25]:
stest_predictions = spark.read.parquet("lab_12/cache/stest_bp_i2i")

In [26]:
stest_predictions.columns

['userId',
 'movieId',
 'rating',
 'n_user_ratings',
 'n_item_ratings',
 'nr_user_cat',
 'nr_item_cat',
 'bp_rating',
 'bp_rating_err',
 'n_used_item_ratings',
 'predicted_i2i_rating',
 'has_i2i_rating']

In [27]:
stest_predictions.count()

3159915

In [28]:
eval_data = stest_predictions.join(stest_als_pred, on=['userId', 'movieId'], how='left')

In [29]:
eval_data.cache()

DataFrame[userId: int, movieId: int, rating: float, n_user_ratings: decimal(21,1), n_item_ratings: decimal(21,1), nr_user_cat: string, nr_item_cat: string, bp_rating: double, bp_rating_err: double, n_used_item_ratings: bigint, predicted_i2i_rating: double, has_i2i_rating: int, predicted_als_rating: float]

In [30]:
def rmse(df, prediction_column):
    mse_df=df.select(F.pow(F.col("rating")-F.col(prediction_column), 2).alias("e2"))\
             .agg(F.avg(F.col("e2")).alias("mse"),
                  F.count(F.col("e2")).alias("n_records"))
    z=mse_df.collect()[0]
    return np.sqrt(z[0]), z[1]

In [31]:
z = rmse(eval_data, "predicted_als_rating")

In [32]:
# (0.83440097064385499, 3154183)
print z

(0.83440097063739527, 3154183)


In [33]:
#top50
#i2i on >2 (0.77253385132439278, 759878)
#als on >2 (0.74363847834562224, 758677)
n=2
z = eval_data.filter(F.col("n_used_item_ratings")>n)
print 'i2i on >',n, rmse(z, "predicted_i2i_rating")
print 'als on >',n, rmse(z, "predicted_als_rating")


i2i on > 2 (0.78208236345341831, 1134944)
als on > 2 (0.75057576694011297, 1133637)


In [34]:
eval_data=eval_data.withColumn("predicted_rating", F.coalesce(F.col("predicted_als_rating"), F.col("bp_rating")))

In [42]:
eval_data=eval_data.withColumn("predicted_rating_avg",
                               F.lit(0.75) * F.col("predicted_rating") +  F.lit(0.25) * F.col("predicted_i2i_rating"))

In [43]:
eval_data.cache()

DataFrame[userId: int, movieId: int, rating: float, n_user_ratings: decimal(21,1), n_item_ratings: decimal(21,1), nr_user_cat: string, nr_item_cat: string, bp_rating: double, bp_rating_err: double, n_used_item_ratings: bigint, predicted_i2i_rating: double, has_i2i_rating: int, predicted_als_rating: float, predicted_rating: double, predicted_rating_avg: double]

In [44]:
n=0
z = eval_data.filter(F.col("n_used_item_ratings")>=n)
#print 'bp on >=',n, rmse(z, "bp_rating")
#print 'i2i on >=',n, rmse(z, "predicted_i2i_rating")
print 'als on >=',n, rmse(z, "predicted_rating")
print 'avg on >=',n, rmse(z, "predicted_rating_avg")

als on >= 0 (0.83495897715420986, 3159915)
avg on >= 0 (0.82227134759083687, 3159915)


In [38]:
#top50
#als on >= 0 (0.83495897716125911, 3159915)
#avg on >= 0 (0.8236655014538885, 3159915)
#top100
#als on >= 0 (0.83495897715420975, 3159915)
#avg on >= 0 (0.82292474871365584, 3159915)

In [39]:
def get_rmse_per_category(eval_data, prediction_column):
    qg = eval_data.withColumn("e2", F.pow(F.col("rating")-F.col(prediction_column), 2))\
             .groupBy(["nr_user_cat", "nr_item_cat"])\
             .agg(F.avg(F.col("e2")).alias("mse"), 
                  F.count(F.col("e2")).alias("n_examples"),
                  F.sum(F.col("has_i2i_rating")).alias("has_i2i_rating"),
                  F.avg(F.col("n_used_item_ratings")).alias("avg_used_item_ratings"))
    epg = qg.toPandas()
    epg['r_examples'] = np.round(epg['n_examples'] * 100. /  epg['n_examples'].sum(), 2)
    epg['rmse'] = np.sqrt(epg['mse'])
    epg['r_has_i2i_rating'] = np.round(epg['has_i2i_rating'] * 100. /  epg['n_examples'], 2)
    
    return epg[["nr_user_cat", "nr_item_cat", 
                "n_examples", "avg_used_item_ratings",
                "r_examples", "rmse", "r_has_i2i_rating"]].sort_values(by='r_examples', ascending=False)
    return epg

In [40]:
get_rmse_per_category(eval_data, "predicted_rating").head(4)

,nr_user_cat,nr_item_cat,n_examples,avg_used_item_ratings,r_examples,rmse,r_has_i2i_rating
0,unr_51,inr_101,2016442,3.416021,63.81,0.783283,82.88
8,unr_06_50,inr_101,900019,0.822118,28.48,0.882012,44.23
3,unr_01_05,inr_101,122519,0.088190,3.88,1.173410,8.12
4,unr_51,inr_11_100,88496,4.161951,2.80,0.823858,78.00


In [41]:
get_rmse_per_category(eval_data, "predicted_rating_avg").head(4)

,nr_user_cat,nr_item_cat,n_examples,avg_used_item_ratings,r_examples,rmse,r_has_i2i_rating
0,unr_51,inr_101,2016442,3.416021,63.81,0.774857,82.88
8,unr_06_50,inr_101,900019,0.822118,28.48,0.873914,44.23
3,unr_01_05,inr_101,122519,0.088190,3.88,1.119163,8.12
4,unr_51,inr_11_100,88496,4.161951,2.80,0.793223,78.00
